In [1]:
import os
import subprocess
import pandas as pd
import shutil

### Execute sub process


In [2]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
class ExecuteSubProcess:
    def __init__(self, backend_manager=None):
        self.backend_manager = backend_manager
    
    def process_return_codes_from_subprocesses(self, ret, msg=""):
        """return code 0: successful execution of the subprocess
        return code -1: Unsuccessful execution of the subprocess
        Subprocess error code is the return code from the subprocess"""
        if ret == 0:
            print(bcolors.OKGREEN + "Success: {}".format(msg) + bcolors.ENDC)
            return 0
        else:
            print(bcolors.FAIL + "Error: {}\nSub process error code: {}".format(msg, ret) + bcolors.ENDC)
            return -1  
        
    def execute_subprocess_local(self, call, msg=""):
        """call a python sub process"""
        print(call)
        completed_proccess = subprocess.run(call)
        ret = completed_proccess.returncode
        return self.process_return_codes_from_subprocesses(ret, msg)
    
    def execute_subprocess_backend(self, cmd, msg=""):
        """submit a backend command to a backend manager"""
        if self.backend_manager:
            stdout, ret = self.backend_manager.run_cmd(cmd)
            print(stdout.readlines())
            return self.process_return_codes_from_subprocesses(ret, msg)
        else:
            print(bcolors.FAIL + "No backend manager" + bcolors.ENDC)
            return -1
    

### Runner

Calls a script file in a specified directory.

In [3]:
class RunnerVariants(ExecuteSubProcess):
    def __init__(self,operand_sizes, 
                 script_dir,
                 threads=4,
                 backend_manager=None,
                 backend_commands=None):
        """
        Runners are objects that executes an external command (such as calling a python or julia or bash script)
        
        RunnerVariants class calls scripts that does the following:
            1. Call a script that generates code for variant algorithms.
            2. Call a script that executes the variant algorithms and outputs a csv file consisting of time measurements
            3. Call a script that generates another script which defines cretain measurement strategy 
            (such as measuring only a subset of algorithms.)

        """
        super().__init__(backend_manager)
        
        self.script_dir = script_dir
        self.operand_sizes = operand_sizes
        self.operands_dir_name = "_".join(self.operand_sizes)
        self.operands_dir = os.path.join(self.script_dir,
                                     "experiments",
                                     self.operands_dir_name)

        self.backend_commands = backend_commands
        self.threads = threads
        # In order to check the status of jobs submitted to a bacth system, 
        # job name should match with the job name mentioned in the bacth script
        self.job_name = "{}_T{}".format(self.operands_dir_name, self.threads)


    def generate_variants_for_measurements(self, generation_script):
        """calls a script that generates variants"""
        script_path = os.path.join(self.script_dir, generation_script)
        args = self.operand_sizes + ["--threads={}".format(self.threads)]
        msg = "{} run: Generate variants"
        if not self.backend_manager:
            call = ["python", script_path] + args
            ret = self.execute_subprocess_local(call, msg.format("Local"))
            return ret
        else:
            cmd = self.backend_commands.build_cmd("python", script_path, " ".join(args))
            ret = self.execute_subprocess_backend(cmd, msg.format("Backend interactive"))
            return ret

    def measure_variants(self, app, runner_script, submit_cmd=None):
        """
        executes a script that measures the variants and generates an output file.txt
        The measurement can be done either locally, in the backend interactively or submnitted 
        to a batch system in the backend.
        """
        runner_path = os.path.join(self.operands_dir, runner_script)
        msg = "{machine} run: "+"Measurements from {script}".format(script=runner_script)
        if not self.backend_manager:
            if os.path.exists(runner_path):
                print("Running Measurements locally")
                call = [app, runner_path]
                ret = self.execute_subprocess_local(call, msg.format(machine="Local"))
                return ret
            else:
                print(bcolors.FAIL + "File not found: "+ msg.format(machine="Local") + bcolors.ENDC)
                return -1
        else:
            if not submit_cmd:
                print("Running Measurements Backend interactive")
                cmd = self.backend_commands.build_cmd(app, runner_path)
                ret = self.execute_subprocess_backend(cmd, msg.format(machine="Backend interactive"))
                return ret
            else:
                print("Running Measurements Backend batch")
                cmd = self.backend_commands.submit_job_cmd(submit_cmd, app, runner_path)
                ret = self.execute_subprocess_backend(cmd, msg.format(machine="Backend batch"))
                return ret

    
    def generate_measurements_script(self, generate_measurement_script, variants, run_id, reps):
        script_path = os.path.join(self.operands_dir, generate_measurement_script)
        cmd_args = "--algs {algs} --rep {rep} --threads {threads} --id {run_id}".format(algs=" ".join(variants),
                                                                                        rep=reps,
                                                                                        threads=self.threads,
                                                                                        run_id=run_id)
        msg = "{machine} run: " + "Generate Measurement script {run_id}".format(run_id=run_id)
        if not self.backend_manager:
            if os.path.exists(script_path):
                call = ["python", script_path] + cmd_args.split()
                ret = self.execute_subprocess_local(call, msg.format(machine="Local"))
                return ret
            else:
                print(bcolors.FAIL + "File not found: "+ msg.format(machine="Local") + bcolors.ENDC)
                return -1
        else:
            cmd = self.backend_commands.build_cmd("python", script_path, cmd_args)
            ret = self.execute_subprocess_backend(cmd, msg.format(machine="Backend interactive"))
            return ret           

    def clean(self):
        """remove arguments folder"""
        if not self.backend_manager:
            if os.path.exists(self.operands_dir):
                shutil.rmtree(self.operands_dir)
            else:
                print(bcolors.FAIL + "Folder not found: {}".format(self.operands_dir) + bcolors.ENDC)
                return -1
        else:
            cmd = "rm -rf {}".format(self.operands_dir)
            ret = self.execute_subprocess_backend(cmd, msg="Removing directory {}".format(self.operands_dir))
            return ret



### Local measurements

In [4]:
operand_sizes = ["75","75","6","75","75"]
script_dir = "sample_generation/"
runner_local = RunnerVariants(operand_sizes, script_dir)

#### 1. Generate variants

In [5]:
runner_local.generate_variants_for_measurements(generation_script="generate-variants-linnea.py")

['python', 'sample_generation/generate-variants-linnea.py', '75', '75', '6', '75', '75', '--threads=4']
New solution:.............2.02e+05
No further generation steps possible.
----------------------------------
Number of nodes:                 8
Solution nodes:                  1
Data:                     1.78e+04
Best solution:            2.02e+05
Intensity:                    11.4
Number of algorithms:            6
Generated Variants.
Success: Local run: Generate variants


0

#### 2. Measure variants (default)

In [6]:
runner_local.measure_variants(app="julia", runner_script="runner.jl")

Running Measurements locally
['julia', 'sample_generation/experiments/75_75_6_75_75/runner.jl']
Success: Local run: Measurements from runner.jl


0

#### 3. Generate custom measurement script

In [7]:
measurements_script = "generate-measurements-script.py"
variants = ['algorithm0', 'algorithm1']
reps = 3
run_id = 0

In [8]:
runner_local.generate_measurements_script(measurements_script, variants, run_id, reps)

['python', 'sample_generation/experiments/75_75_6_75_75/generate-measurements-script.py', '--algs', 'algorithm0', 'algorithm1', '--rep', '3', '--threads', '4', '--id', '0']
Success: Local run: Generate Measurement script 0


0

#### 4. Measure variants (by calling the custom script)

In [9]:
runner_competing_script = "runner_competing_0.jl"
runner_local.measure_variants(app="julia", runner_script=runner_competing_script)

Running Measurements locally
['julia', 'sample_generation/experiments/75_75_6_75_75/runner_competing_0.jl']
Success: Local run: Measurements from runner_competing_0.jl


0

#### 5. Remove the directories generated by the runner

This removes the generated variants and the measurements

In [10]:
runner_local.clean()

### Backend measurements

In [11]:
from backend_manager import BackendManager,Commands

#### 1. Instantiate a backend manager

In [12]:
bm = BackendManager(server="login18-1.hpc.itc.rwth-aachen.de", uname="as641651")
bm.connect()
cmds = Commands(source="~/.analyzer")

### Interactive 

#### 2. Generate variants interactively 

In [13]:
operand_sizes = ["75","75","6","75","75"]
script_dir = "sample_generation/"
generation_script = "generate-variants-linnea.py"
runner = RunnerVariants(operand_sizes, script_dir,backend_manager=bm, backend_commands=cmds)

In [14]:
ret = runner.generate_variants_for_measurements(generation_script=generation_script)

source ~/.analyzer; cd sample_generation; python generate-variants-linnea.py 75 75 6 75 75 --threads=4
['New solution:.............2.02e+05\n', 'No further generation steps possible.\n', '----------------------------------\n', 'Number of nodes:                 8\n', 'Solution nodes:                  1\n', 'Data:                     1.78e+04\n', 'Best solution:            2.02e+05\n', 'Intensity:                    11.4\n', 'Number of algorithms:            6\n', 'Generated Variants.\n']
Success: Backend interactive run: Generate variants


In [15]:
ret

0

#### 3. Measure variants interactively

In [16]:
runner.measure_variants(app="julia", runner_script="runner.jl")

Running Measurements Backend interactive
source ~/.analyzer; cd sample_generation/experiments/75_75_6_75_75; julia runner.jl 
[]
Success: Backend interactive run: Measurements from runner.jl


0

#### 4. Generate measurement script interactively

In [17]:
measurements_script = "generate-measurements-script.py"
variants = ['algorithm0', 'algorithm1']
reps = 3
run_id = 0

In [18]:
runner.generate_measurements_script(measurements_script, variants, run_id, reps)

source ~/.analyzer; cd sample_generation/experiments/75_75_6_75_75; python generate-measurements-script.py --algs algorithm0 algorithm1 --rep 3 --threads 4 --id 0
[]
Success: Backend interactive run: Generate Measurement script 0


0

#### 5. Run measurement script interactively

In [19]:
runner_competing_script = "runner_competing_0.jl"
runner.measure_variants(app="julia", runner_script=runner_competing_script)

Running Measurements Backend interactive
source ~/.analyzer; cd sample_generation/experiments/75_75_6_75_75; julia runner_competing_0.jl 
[]
Success: Backend interactive run: Measurements from runner_competing_0.jl


0

### Batch system

#### 6. Generate another measurement script (interactively)

In [20]:
run_id = 1
runner.generate_measurements_script(measurements_script, variants, run_id, reps)

source ~/.analyzer; cd sample_generation/experiments/75_75_6_75_75; python generate-measurements-script.py --algs algorithm0 algorithm1 --rep 3 --threads 4 --id 1
[]
Success: Backend interactive run: Generate Measurement script 1


0

#### 7. Measure variants by submitting to a batch system

In [21]:
submit_cmd = "sbatch submit.sh"
runner_competing_script = "runner_competing_1.jl"
runner.measure_variants(app="julia", runner_script=runner_competing_script, submit_cmd=submit_cmd)

Running Measurements Backend batch
source ~/.analyzer; cd sample_generation/experiments/75_75_6_75_75; sbatch submit.sh julia 'runner_competing_1.jl '
['Submitted batch job 28807909\n']
Success: Backend batch run: Measurements from runner_competing_1.jl


0

#### 8. Check status of the submitted job

In [22]:
bm.check_slrum_status(runner.job_name)

          28807909        ih               75_75_6_75_75_T4 as641651  RUNNING       0:05   3:00:00      1 linuxihdc074



2

#### 9. Check if the required output file from running the measurement script has been generated

In [23]:
bm.check_if_file_exists(os.path.join(runner.operands_dir, "run_times_competing_1.csv"))

True

#### 10. Remove the variants directory and the measurement  files

In [24]:
runner.clean()

rm -rf sample_generation/experiments/75_75_6_75_75
[]
Success: Removing directory sample_generation/experiments/75_75_6_75_75


0